In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import os
import pandas as pd
os.getcwd()
path = 'Reader_Emotion_Seg_no_stopword.csv'
f = pd.read_csv(path)

f.head()

f['concate'] = f['title_seg_no_stw']+f['content_seg_no_stw']

In [0]:
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.layers.merge import concatenate
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import Dense, Embedding
from tensorflow.python.keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D
from tensorflow.python.keras import initializers

In [0]:
def textCNN(texts, labels):
  training_text, test_text, training_label, test_label = train_test_split(texts, labels, test_size=0.1)

  token = Tokenizer(num_words = 20000)
  token.fit_on_texts(training_text)
  vocab = token.word_index

  x_train_seq = token.texts_to_sequences(training_text)
  x_test_seq = token.texts_to_sequences(test_text)
  x_train = sequence.pad_sequences(x_train_seq, maxlen = 150)
  x_test = sequence.pad_sequences(x_test_seq, maxlen = 150)

  y_train = np_utils.to_categorical(training_label)
  y_test = np_utils.to_categorical(test_label)

  num_labels = 8
  main_input = Input(shape=(150),), dtype='float64')
    # pre-train embeddings
    # embedder = Embedding(len(vocab) + 1, 300, input_length = 20, weights = [embedding_matrix], trainable = False)
    # embed = embedder(main_input)

  embedder = Embedding(len(vocab)+1, 300, input_length=150)
  embed = embedder(main_input)

    # filter size, region size
  cnn = Convolution1D(2, 2, padding='same', strides = 1, activation='relu')(embed)
  cnn = MaxPool1D(pool_size=4)(cnn)
  flat = Flatten()(cnn)
  drop = Dropout(0.2)(flat)
  main_output = Dense(num_labels, activation='softmax')(drop)
  model = Model(inputs = main_input, outputs = main_output)
  model.summary()
    
  optmzr = optimizers.Adam(lr=0.001)
  model.compile(loss = 'categorical_crossentropy', optimizer=optmzr, metrics = ['accuracy'])
  train_history = model.fit(x_train, y_train, batch_size = 128, epochs = 100, 
                              verbose = 2, validation_data=(x_test, y_test))
  pre_probability = model.predict(x_test)
  predicted = pre_probability.argmax(axis=-1)

  from sklearn import metrics
  print("Classification report for classifier:\n%s\n"
  % ( metrics.classification_report(test_label, predicted)))

  import matplotlib.pyplot as plt
  plt.plot(train_history.history['accuracy'])
  plt.plot(train_history.history['val_accuracy'])
  plt.title('Train History')
  plt.ylabel('accuracy')
  plt.xlabel('Epoch')
  plt.legend(['train','validation'], loc = "upper left")
  plt.show()

  plt.plot(train_history.history['loss'])
  plt.plot(train_history.history['val_loss'])
  plt.title('Train History')
  plt.ylabel('loss')
  plt.xlabel('Epoch')
  plt.legend(['train','validation'], loc = "upper left")
  plt.show()

In [0]:
def rnn(texts, labels):
  training_text, test_text, training_label, test_label = train_test_split(texts, labels, test_size=0.1)

  token = Tokenizer(num_words = 20000)
  token.fit_on_texts(training_text)
  print(token.document_count)

  x_train_seq = token.texts_to_sequences(training_text)
  x_test_seq = token.texts_to_sequences(test_text)
  x_train = sequence.pad_sequences(x_train_seq, maxlen = 400)
  x_test = sequence.pad_sequences(x_test_seq, maxlen = 400)

  y_train = np_utils.to_categorical(training_label)
  y_test = np_utils.to_categorical(test_label)

  from keras.models import Sequential
  from keras.layers.core import Dense, Dropout, Activation, Flatten
  from keras.layers.embeddings import Embedding
  from keras.callbacks import ModelCheckpoint

  model = Sequential()
  model.add(Embedding(output_dim=32,input_dim=20000,input_length=300))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(units=256,activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(units=128,activation='relu'))
  model.add(Dense(units=8,activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  # checkpoint
  # filepath="C:/Users/doudi/OneDrive/Documents/TMU-GIDS/Lab/Competition/AI cup 2019/weights.best.hdf5"
  checkpoint= ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
  train_history = model.fit(xtrain, y_train_OneHot, validation_data=(xtest, y_test_OneHot), epochs=100, batch_size=100, callbacks=[checkpoint],
                            verbose=1, validation_split=0.2)